In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install Cython

In [ ]:
!pip install nemo_toolkit['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 28.0 MB/s 

In [ ]:
# !wget -O checkpoint.ckpt "https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020205_vnu_edu_vn/EbYkGgMABVBFg3oiM9iWA3MBnjmxvVIJJ-5BEDUwJ7w7LQ?e=uwmuCM&download=1"

In [ ]:
!wget -O checkpoint.ckpt "https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020108_vnu_edu_vn/Ef7m8t9hPMtPkkmP53dFnpEBXhFsC6HzOIO3bjD_actStw?e=ThGyC6&download=1"

--2023-12-16 18:54:13--  https://vnueduvn-my.sharepoint.com/:u:/g/personal/21020108_vnu_edu_vn/Ef7m8t9hPMtPkkmP53dFnpEBXhFsC6HzOIO3bjD_actStw?e=ThGyC6&download=1
Resolving vnueduvn-my.sharepoint.com (vnueduvn-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to vnueduvn-my.sharepoint.com (vnueduvn-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/21020108_vnu_edu_vn/Documents/checkpoint.ckpt?ga=1 [following]
--2023-12-16 18:54:15--  https://vnueduvn-my.sharepoint.com/personal/21020108_vnu_edu_vn/Documents/checkpoint.ckpt?ga=1
Reusing existing connection to vnueduvn-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 152292670 (145M) [application/octet-stream]
Saving to: ‘checkpoint.ckpt’

checkpoint.ckpt     100%[===================>] 145.24M  39.6MB/s    in 6.5s    

2023-12-16 18:54:22 (22.3 MB/s) - ‘checkpoint.ckpt’ saved [152292670/152292670]



In [ ]:
import os
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML

# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import DictConfig
import copy
import pytorch_lightning as pl
import torch

In [ ]:
model = nemo_asr.models.EncDecCTCModel.load_from_checkpoint("checkpoint.ckpt", map_location='cpu')

[NeMo W 2023-12-16 18:54:48 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /work/hpc/potato/asr/data/LibriSpeech/dev-clean_manifest.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-12-16 18:54:48 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_mult

[NeMo I 2023-12-16 18:54:48 features:289] PADDING: 16


In [ ]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install flask[async]
!ngrok authtoken 2ZDpxzhxZP7h0QMThOw4FJkJrIS_3DCv44LHYHKX1jNNMzVn8

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=62728f13acf9b9b4522d93b81269ecd614e954331b6dd7b79f0257d40913860c
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
def get_result(path2audio_files) :
    return model.transcribe(paths2audio_files=path2audio_files, batch_size=len(path2audio_files))

In [ ]:
# fuck_files = ['flask_fuck.wav', 'flask_fuck - Copy.wav']
# print(get_result(fuck_files))

In [ ]:
!apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 24 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 amd64 14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1 [240 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libs

In [ ]:
!pip install aiohttp[speedups]
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 13.4 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
import subprocess
from io import StringIO

In [ ]:
# async def post_request() :
#   async with session:
#     response = await session.post(url="/", data={"key": "value"})
#     print(await response)

In [ ]:
# import asyncio
# import aiohttp
# from aiohttp import web

# import nest_asyncio
# nest_asyncio.apply()

# routes = web.RouteTableDef()

# @routes.get('/')
# async def hello(request):
#     return web.Response(text="Hello, world")

# app = web.Application()
# app.add_routes(routes)
# web.run_app(app)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
async def home():
    return """
        <h1>Please choose the audio file:</h1>
        <h3>Allowed audio file types: .wav</h3>
        <form action="/uploader" method="POST" enctype = "multipart/form-data">
            <input type = "file" name = "file" multiple/>
            <br>
            <input type = "submit"/>
        </form>
        <br>
        <p id="return"></p>
        """

def process_file(files) :
      path = []
      duration = []
      for file in files:
          parts = file.filename.split(".")
          if parts[-1] != "wav":
              return "Not wav file"
          local_filename = "flask_" + file.filename
          print(local_filename)
          file.save(local_filename)
          path.append("/content/" + local_filename)
          output = float(subprocess.check_output(['sox', '--i', '-D', local_filename]))
          duration.append(output)
      output = get_result(path)
      ans = 'file uploaded successfully <br>'
      for idx in range(len(output)):
          ans = ans + "{text: " +  output[idx] + ", duration: " + str(duration[idx]) + "}" +  "<br>"

      return ans

import time

@app.route('/uploader', methods = ['GET', 'POST'])
async def upload_file():
   if request.method == 'POST':
      # async with aiohttp.ClientSession("/") as session:
      #     async with session.get('/') as resp:
      #         print(resp.status)
      #         print(await resp.text())
      start_time = time.time()
      f = request.files.getlist('file')
      ans = process_file(f)
      duration = time.time() - start_time
      ans = ans + "<br>Duration: " + str(duration) + "s."
      return ans

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5bf9-34-23-116-4.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 18:56:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2023 18:56:25] "GET /favicon.ico HTTP/1.1" 404 -


flask_test10p.wav


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Exception on /uploader [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/usr/local/lib/python3.10/dist-packages/asgiref/sync.py", line 277, in __call__
    return call_result.result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 451, in result
    return self.__get_result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._exception
  File "/u

Note:
- limit batch
- ?